The transfer function of the causal LTI system with a realization $(A, B, C, D)$ is given by
\begin{equation*}
    H(s) = C(sI_{n} - A)^{-1}B.
\end{equation*}
Notice that $H(s) = Cv(s)$, where $v(s)$ is the solution of a parametrized linear coercive model
\begin{equation}
    a(v, w; s) = l(w),
\end{equation}
where $a(v, w; s) = w^{*}(sI_{n} - A)v$ and $l(w) = w^{*}B$. 

We replace the matrix computation of $(sI_{n} - A)^{-1}B$ with a parametrized model (1), and the transfer function $H(s)$ is the output of the model, with $C$ serving as the output operator.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from pymor.basic import *
from pymor.algorithms.to_matrix import to_matrix
from pymor.vectorarrays.numpy import NumpyVectorSpace
from pymor.operators.numpy import NumpyMatrixOperator
from pymor.operators.constructions import LincombOperator
from pymor.parameters.functionals import ProjectionParameterFunctional

In [12]:
def MatrixModel(A, B, C):

    '''
    Description
    -----------
    This function creates a StationaryModel for the following linear coercive model derived from the given three matrices A, B, and C:
    
        a(v, w; s) = w*(sI_n - A)v and l(w) = w*B.

    Parameters
    ----------
    A: NumpyMatrixOperator or numpy.ndarray
        A.shape = (n, n) or to_matrix(A).shape = (n, n)
    B: NumpyMatrixOperator or numpy.ndarray
        A.shape = (n, 1) or to_matrix(B).shape = (n, 1)
    C: NumpyMatrixOperator or numpy.ndarray
        A.shape = (1, n) or to_matrix(C).shape = (1, n)

    Output
    ------
    model_TF: StationaryModel
    '''

    # Define operators (and also a dimension of a model)
    if isinstance(A, np.ndarray):
        dim = A.shape[0]
        A_op = NumpyMatrixOperator(A)
    else:
        dim = to_matrix(A).shape[0]
        A_op = A

    if isinstance(B, np.ndarray):
        B_op = NumpyMatrixOperator(B) 
    else:
        B_op = B  

    if isinstance(C, np.ndarray):
        C_op = NumpyMatrixOperator(C)  
    else:
        C_op = C

    I_op = NumpyMatrixOperator(np.eye(dim))
    
    # Define parameter functional for 's'
    s_param = ProjectionParameterFunctional('s', 1)

    # Define bilinear form a(v, w; s) = w*(sI - A)v
    a_op_1 = LincombOperator([I_op, A_op], [s_param, -1])

    # Define linear functional l(w) = w^*B
    l_op_1 = B_op

    # Define the StationaryModels
    model_TF = StationaryModel(operator=a_op_1, rhs=l_op_1, output_functional = C_op)

    return model_TF


In [32]:
# StationaryModel constructed using random numpy arrays
np.random.seed(127)
matrixA = np.random.rand(20, 20)
matrixB = np.random.rand(20).reshape(20,1)
matrixC = np.random.rand(20).reshape(1,20)

# StationaryModel
model_TF = MatrixModel(A = matrixA, B = matrixB, C = matrixC)
model_TF

StationaryModel(
    LincombOperator(
        (NumpyMatrixOperator(<20x20 dense>), NumpyMatrixOperator(<20x20 dense>)),
        (ProjectionParameterFunctional('s', index=0), -1)),
    NumpyMatrixOperator(<20x1 dense>),
    output_functional=NumpyMatrixOperator(<1x20 dense>),
    products={},
    output_d_mu_use_adjoint=True)

In [37]:
# Set parameter for evaluation
parameter = {'s': 1.4 + 2*1j}  # s = 1.4 + 2i

# Solve the model - (sI_n - A)^{-1}B
solution = model_TF.solve(parameter).to_numpy()

# Get the output - C(sI_n - A)^{-1}B
output = model_TF.output(parameter)

# Exact transfer function
exact = (matrixC@np.linalg.inv(parameter['s']*np.eye(20) - matrixA)@matrixB)

# A comparison between the model's result and the exact computation
print(f'The output of the model is {output[0,0]}.')
print(f'The exact value of a transfer function H{parameter["s"]} is {exact[0,0]}.')
print(f'|H(s) - H_m(s)| is {abs(exact - output)[0,0]}.')

Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))

The output of the model is (-0.5691965467381752-0.13555671118980517j).
The exact value of a transfer function H(1.4+2j) is (-0.5691965467381752-0.135556711189805j).
|H(s) - H_m(s)| is 1.6653345369377348e-16.


In [38]:
# StationaryModel constructed using matrices obtained from penzl example
from pymor.models.examples import penzl_example

penzl = penzl_example()

model_penzl = MatrixModel(penzl.A, penzl.B, penzl.C)
model_penzl

StationaryModel(
    LincombOperator(
        (NumpyMatrixOperator(<1006x1006 dense>), NumpyMatrixOperator(<1006x1006 sparse, 1012 nnz>)),
        (ProjectionParameterFunctional('s', index=0), -1)),
    NumpyMatrixOperator(<1006x1 dense>),
    output_functional=NumpyMatrixOperator(<1x1006 dense>),
    products={},
    output_d_mu_use_adjoint=True)

In [45]:
# Set parameter for evaluation
parameter = {'s': -1 + 2*1j}  # s = - 1 + 2i

# Solve the model - (sI_n - A)^{-1}B
solution = model_penzl.solve(parameter).to_numpy()

# Get the output - C(sI_n - A)^{-1}B
output = model_penzl.output(parameter)

# Exact transfer function
A = to_matrix(penzl.A).toarray()
B = to_matrix(penzl.B)
C = to_matrix(penzl.C)

exact = (C@np.linalg.inv(parameter['s']*np.eye(penzl.order) - A)@B)

# A comparison between the model's result and the exact computation
print(f'The output of the model is {output[0,0]}.')
print(f'The exact value of a transfer function H{parameter["s"]} is {exact[0,0]}.')
print(f'|H(s) - H_m(s)| is {abs(exact - output)[0,0]}.')

Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))

The output of the model is (6.192665682271828-1.7662892159754267j).
The exact value of a transfer function H(-1+2j) is (6.1926656822718265-1.7662892159754267j).
|H(s) - H_m(s)| is 1.7763568394002505e-15.


### POD-Galerkin RB method

As an example, we will use `penzl_example` from `pymor.models.examples` to construct a reduced basis.

In [46]:
from pymor.models.examples import penzl_example

penzl = penzl_example()
model_penzl = MatrixModel(penzl.A, penzl.B, penzl.C)

In [47]:
# Define a parameter space
parameter_space = model_penzl.parameters.space(0.01, 10.)

# Define a training set
training_set = parameter_space.sample_randomly(50)

# Compute FOM solutions for the parameters in the training set
solution_snapshots = model_penzl.solution_space.empty()
for s in training_set:
    solution_snapshots.append(model_penzl.solve(s))

# Snapshot matrix S
snapshot_matrix = solution_snapshots.to_numpy().T
print(f'An {snapshot_matrix.shape[0]} by {snapshot_matrix.shape[1]} snapshot matrix is \n {snapshot_matrix}')

Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))

An 1006 by 50 snapshot matrix is 
 [[ 0.10791712  0.10508859  0.10858926 ...  0.1037334   0.10717139
   0.10234687]
 [-0.09056608 -0.09433111 -0.08958912 ... -0.09596436 -0.09161048
  -0.0975374 ]
 [ 0.05208595  0.05131127  0.05227672 ...  0.05095332  0.05187756
   0.05059421]
 ...
 [ 0.00099429  0.00099761  0.00099346 ...  0.00099911  0.0009952
   0.00100059]
 [ 0.0009933   0.00099662  0.00099247 ...  0.00099811  0.00099421
   0.00099959]
 [ 0.00099232  0.00099562  0.00099149 ...  0.00099712  0.00099322
   0.0009986 ]]


In [56]:
# Finding the Singular Value Decomposition (SVD) of the snapshot matrix -> S = UΣV^T
U, D, Vt = np.linalg.svd(snapshot_matrix, full_matrices = True) 
# using 'pod_basis, pod_singular_values = pod(solution_snapshots)' gives incomplete result; that's why we don't use it here. 

# Activate below to automatically select the number of modes based on the energy criterion
'''
The number of modes m should be chosen based on the energy criterion, which ensures that a sufficient 
portion of the system's total energy (or variance) is captured by the first m modes.
'''
#cumulative_energy = np.cumsum(D**2) / np.sum(D**2)

#Select the number of modes m to capture at least 95% of the energy
#threshold = 0.95  # 95% of the total energy
#m = np.argmax(cumulative_energy >= threshold) + 1

m = 20 # this is also a reduction order

if m > min(snapshot_matrix.shape):
    raise ValueError("m cannot exceed the rank of the snapshot matrix.")

# The reduced basis (POD basis)
pod_basis_numpy = U[:,:m]

# Convert NumPy array into VectorArray 
space = NumpyVectorSpace(model_penzl.order)
pod_basis = space.make_array(pod_basis_numpy.T) #This is actually transpose of POD-RB basis

print(f'The reduced basis (containing the first {m} left singular vectors (POD modes) of the snapshot matrix as its columns) is \n {pod_basis_numpy}')

The reduced basis (containing the first 20 left singular vectors (POD modes) of the snapshot matrix as its columns) is 
 [[-0.19696096 -0.31340348  0.36198988 ... -0.00431519 -0.00049817
   0.00223207]
 [ 0.17630327  0.23589406 -0.16498272 ... -0.0379153  -0.02853059
   0.00898774]
 [-0.09617923 -0.14845524  0.1618159  ...  0.36862449  0.29847176
  -0.06855028]
 ...
 [-0.0018688  -0.00275277  0.0026819  ... -0.0157844   0.00189082
  -0.00276168]
 [-0.00186694 -0.00275005  0.00267931 ... -0.00445315 -0.0062538
  -0.00233055]
 [-0.00186508 -0.00274733  0.00267672 ...  0.00197302 -0.00953304
   0.00471574]]


In [57]:
from pymor.reductors.basic import StationaryRBReductor

# POD-Galerkin RB method
pod_reductor = StationaryRBReductor(model_penzl, RB = pod_basis) 
pod_rom = pod_reductor.reduce()
pod_rom

Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))

StationaryModel(
    LincombOperator(
        (NumpyMatrixOperator(<20x20 dense>), NumpyMatrixOperator(<20x20 dense>)),
        (ProjectionParameterFunctional('s', index=0), -1)),
    NumpyMatrixOperator(<20x1 dense>),
    output_functional=NumpyMatrixOperator(<1x20 dense>),
    products={},
    output_d_mu_use_adjoint=True,
    name='StationaryModel_reduced')